# Case Study: Neuro

## Import Packages

In [13]:
%load_ext autoreload
%autoreload 2

import os
import sys
import glob
import pandas as pd
import math
import matplotlib.pyplot as plt
import datetime
# import from absolute directory
srcdir = '/Users/pengzhizhang/cellDancer/src'
srcdir = '/Users/shengyuli/Library/CloudStorage/OneDrive-HoustonMethodist/work/Velocity/bin/cellDancer_polish/src'
srcdir = '/Users/wanglab/Documents/ShengyuLi/Velocity/bin/celldancer_polish/src'
#srcdir = '/Users/chen.weiqing/Documents/GitHub/cellDancer/src'
sys.path.append(srcdir)
from celldancer import velocity_estimation
from celldancer.compute_cell_velocity import compute
from celldancer.pseudo_time import pseudo_time
import celldancer.cdplt as cdplt
from celldancer.cdplt import colormap

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Velocity Estimation for Sample Genes

In [3]:
%autoreload 2

raw_path="/Users/shengyuli/Library/CloudStorage/OneDrive-HoustonMethodist/work/Velocity/data/raw_data/DentateGyrus.csv"
raw_path='/Users/wanglab/Documents/ShengyuLi/Velocity/data/raw_data/DentateGyrus.csv'
#raw_path='/Users/chen.weiqing/code/celldancer/DentateGyrus.csv'
cell_type_u_s=pd.read_csv(raw_path)

In [ ]:
%autoreload 2

enu=1
n_jobs_list=[40,35,30,25,20,15,10,5,1]

for enu,n_jobs in enumerate(n_jobs_list):
    time_start=datetime.datetime.now()

    max_epoches=200
    check_val_every_n_epoch=10
    patience=3
    permutation_ratio=0.1
    speed_up=True
    norm_u_s=False
    norm_cell_distribution=True

    # gene_list=['Ntrk2','Dctn3','Nav1','Psd3',\
    #            'Gnao1','Ncald','Klf7','Ank2',\
    #            'Evl','Cdh13','Ppfia2','Astn1',\
    #            'Nrxn1','Dcx','Prkcg','Pygb','Scn2a1',\
    #             'Tbc1d16','Slc1a3','Syt11']


    #gene_list=list(load_raw_data.gene_name.drop_duplicates())
    gene_list=None
    #result_path = '/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/tunning_train'
    #result_path=None

    folder_name=("epoch"+str(max_epoches)+
                 "check_n"+str(check_val_every_n_epoch)+
                 "Ratio"+str(permutation_ratio)+
                 "autoD"+str(speed_up)+
                 "autoN"+str(norm_u_s)+
                 "p"+str(patience)+
                 "bin"+str(norm_cell_distribution)+
                'job'+str(n_jobs))

    save_path = '/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/'+folder_name+'/'

    if not os.path.isdir(save_path):
        os.mkdir(save_path)

    # Velocity Estimation
    loss_df, cellDancer_df=velocity_estimation.velocity(cell_type_u_s,\
                                                     gene_list=gene_list,\
                                                     max_epoches=max_epoches,\
                                                     check_val_every_n_epoch=check_val_every_n_epoch,\
                                                     patience=patience,\
                                                     permutation_ratio=permutation_ratio,\
                                                     speed_up=speed_up,\
                                                     norm_u_s=norm_u_s,\
                                                     norm_cell_distribution=norm_cell_distribution,\
                                                     n_jobs=n_jobs,\
                                                     save_path=save_path)
    time_end=datetime.datetime.now()
    time_spent=(time_end-time_start).seconds/60
    time_report=pd.DataFrame({'n_jobs':n_jobs,'time_start':time_start,'time_end':time_end,'time_spent(min)':time_spent}, index=[enu])
    if enu==0:
        time_report.to_csv('/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/test_speed.csv',index=False)
    else:
        time_report.to_csv('/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/test_speed.csv',index=False,mode='a',header=False)



Using /Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/epoch200check_n10Ratio0.1autoDTrueautoNFalsep3binTruejob40/cell_dancer_velocity_2022-05-26 19-09-50 as the output path.


Velocity Estimation:   6%|█▎                     | 3/54 [01:33<25:57, 30.54s/it]

In [10]:
cellDancer_df

,cellIndex,gene_name,s0,u0,s1,u1,alpha,beta,gamma,loss,cellID,clusters,embedding1,embedding2
0,0,Klf7,1.294797,0.408467,1.452581,0.478509,0.499887,0.880862,0.034163,0.081968,10X83_2:AAACGGGGTCTCTTTAx,ImmGranule2,18.931086,-1.862429
1,1,Klf7,1.256870,0.379136,1.401708,0.454003,0.483179,0.879489,0.034826,0.081968,10X83_2:AACCATGGTTCAACCAx,ImmGranule2,18.419891,-1.282895
2,2,Klf7,3.395004,0.893599,3.809754,0.889776,0.852186,0.962212,0.008934,0.081968,10X83_2:AACACGTTCTGAAAGAx,CA2-3-4,2.369887,16.868419
3,3,Klf7,2.739187,0.640505,3.023452,0.682891,0.690913,0.946351,0.013732,0.081968,10X83_2:AAAGATGCATTGAGCTx,CA,-5.351040,10.676485
4,4,Klf7,2.433943,0.662303,2.722881,0.696757,0.687945,0.934673,0.016911,0.081968,10X83_2:AACCATGTCTACTTACx,CA1-Sub,-6.189126,11.754900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362795,18135,Prkcg,0.007313,0.015161,0.010244,0.097734,0.174282,0.602637,0.447664,0.092245,10X84_3:TTTCCTCCACCATCCTx,ImmGranule1,10.812611,-2.487668
362796,18136,Prkcg,0.861493,0.594831,0.845331,0.582430,0.351807,0.633135,0.474680,0.092245,10X84_3:TTTGTCACATGAAGTAx,CA2-3-4,8.246204,23.482788
362797,18137,Prkcg,0.015733,0.016774,0.017266,0.098419,0.173408,0.603174,0.448147,0.092245,10X84_3:TTTCCTCCACGGTAAGx,nIPC,-3.441272,-4.917364
362798,18138,Prkcg,0.034927,0.057718,0.044480,0.140709,0.200725,0.601927,0.447669,0.092245,10X84_3:TTTGTCAAGCGTCAAGx,ImmGranule2,16.394199,-6.143549
